In [ ]:
from boto3 import client
import json
import xmltodict
import pickle

In [ ]:
data=open('../config.json', 'r').read()
config = json.loads(data)['hitCreation']

In [ ]:
if config['production']:
    endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'
else:
    endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'
print(config['production'])
print(config['numAssignments'])

In [ ]:
cl = client('mturk',
    aws_access_key_id = "xxxx",
    aws_secret_access_key = "xxxx",
    region_name='us-east-1',
    endpoint_url = endpoint_url
    )

In [ ]:
#cl.get_account_balance()

function for creating the HIT

In [ ]:
def create_hit():
    if config['production']:
        quals = [
           {
               'QualificationTypeId': '00000000000000000071',
               'Comparator': 'EqualTo',
               'LocaleValues': [{
                   'Country': 'US',
               }],
           },
           {
               'QualificationTypeId': '000000000000000000L0',
               'Comparator': 'GreaterThanOrEqualTo',
               'IntegerValues': [
                   97
                   ],
           },
           {
               'QualificationTypeId': '00000000000000000040',
               'Comparator': 'GreaterThanOrEqualTo',
               'IntegerValues': [
                   100
                   ],
           },
        ]
    else:
        quals=[]
    questionText = "<ExternalQuestion xmlns=\"http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/"
    questionText += "2006-07-14/ExternalQuestion.xsd\">\n<ExternalURL>" + config['taskUrl']
    questionText += "</ExternalURL>\n  <FrameHeight>700</FrameHeight>\n</ExternalQuestion>"
    new_hit=cl.create_hit(
        MaxAssignments=config['numAssignments'],
        AutoApprovalDelayInSeconds=604800,
        LifetimeInSeconds=config['lifetime'],
        AssignmentDurationInSeconds=config['duration'],
        Reward=config['rewardAmount'],
        Title=config['title'],
        Keywords=config['keywords'],
        Description=config['description'],
        Question=questionText,
        QualificationRequirements=quals,
        )
    hit_id=new_hit['HIT']['HITId']
    return(new_hit)

Creation of the HIT

In [ ]:
url=create_hit()
print("A new HIT has been created. You can preview it here:")
if config['production']:
    print("https://worker.mturk.com/mturk/preview?groupId=" + url['HIT']['HITGroupId'])
else:
    print("https://workersandbox.mturk.com/mturk/preview?groupId=" + url['HIT']['HITGroupId'])

print ("HITID = " + url['HIT']['HITId'] + " (Use to Get Results)")

Update the expiration time of a HIT

In [ ]:
cl.update_expiration_for_hit(HITId='xxxx',ExpireAt=1582806900)

Results of the workers

In [ ]:
worker_results = cl.list_assignments_for_hit(HITId=url['HIT']['HITId'], AssignmentStatuses=['Submitted'])
print(worker_results)

In [ ]:
worker_results = cl.list_assignments_for_hit(HITId='xxxx', AssignmentStatuses=['Submitted'])
worker_results

Save the results in a obj file

In [ ]:
file= open('results.obj', 'wb')
pickle.dump(worker_results, file)

In [ ]:
import os
os.path.getsize('results.obj')

Reads the file

In [ ]:
#file1=open('results.obj', 'rb')
#workers=pickle.load(file1)

In [ ]:
HITTypeId= 'xxxx'
cl.list_reviewable_hits(HITTypeId=HITTypeId)

Approves the assignments. It is not necessary. It will be done automatically after a while from the submission of the answers

In [ ]:
cl.approve_assignment(AssignmentId='xxx',RequesterFeedback="Thank you!")

To appprove an assignment that you previously rejected:

In [ ]:
cl.approve_assignment(AssignmentId='xxx',RequesterFeedback="Thank you!",OverrideRejection=True)

Reject the output of a worker

In [ ]:
cl.reject_assignment(AssignmentId='xxx',RequesterFeedback="You failed the control questions. We are sorry.")

Lists the last HITs published on the Marketplace

In [ ]:
cl.list_hits(MaxResults=1)